# Open Shell Code tests


Broadly, there are three things we need to do make sure of before we can include the open shell code into the main repo.
1. Closed shell code is not affected in any way by the implementation of open shell.
2. Open shell code replicates closed shell results accurately.
3. Open shell results are accurate.

## 1.

We can check this using the existing tests for closed shell code.

## 2.

Overall
- Run closed shell experiemnts with both open and closed shell methods.
- Hamiltonians are numerically close

Localizers
- Active MOs should be consistent across un/restricted for closed shells.
- 

In [1]:
from nbed.driver import NbedDriver
from pathlib import Path

mol_filepath = Path("molecules/water.xyz").absolute()

args = {
    "geometry": str(mol_filepath),
    "n_active_atoms": 1,
    "basis": "STO-3G",
    "xc_functional": "b3lyp",
    "projector": "mu",
    "localization": "spade",
    "convergence": 1e-8,
    "savefile": None,
    "run_ccsd_emb": False,
    "run_fci_emb": False,
}

restric_driver = NbedDriver(
    geometry=args["geometry"],
    n_active_atoms=args["n_active_atoms"],
    basis=args["basis"],
    xc_functional=args["xc_functional"],
    projector=args["projector"],
    localization=args["localization"],
    convergence=args["convergence"],
    savefile=args["savefile"],
    run_ccsd_emb=args["run_ccsd_emb"],
    run_fci_emb=args["run_fci_emb"],
    force_unrestricted=False,
)

unrestric_driver = NbedDriver(
    geometry=args["geometry"],
    n_active_atoms=args["n_active_atoms"],
    basis=args["basis"],
    xc_functional=args["xc_functional"],
    projector=args["projector"],
    localization=args["localization"],
    convergence=args["convergence"],
    savefile=args["savefile"],
    run_ccsd_emb=args["run_ccsd_emb"],
    run_fci_emb=args["run_fci_emb"],
    force_unrestricted=True,
)



/home/mwilliams/.cache/pypoetry/virtualenvs/nbed-T5bcl2YH-py3.8/lib/python3.8/site-packages/pyscf/lib/misc.py:46: H5pyDeprecationWarning: Using default_file_mode other than 'r' is deprecated. Pass the mode to h5py.File() instead.
  h5py.get_config().default_file_mode = 'a'
/home/mwilliams/.cache/pypoetry/virtualenvs/nbed-T5bcl2YH-py3.8/lib/python3.8/site-packages/_distutils_hack/__init__.py:30: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


## Full system Hamiltonians should match global FCI energy.

Note that we use the energy from the global fci and the Molecular Orbital coefficients from the KS. The fci object does not have the correct structure for us to make a hamiltonian from it but the underlying HF object which both the FCI and KS objects use to construct MOs should give very similar orbitals, such that our two methods of exact solution match.

In [2]:
from nbed.ham_builder import HamiltonianBuilder
from openfermion import QubitOperator, get_sparse_operator, count_qubits
import scipy as sp

# First check restricted
# This uses the spinorb_from_spatial function defined by openfermion
rks = restric_driver._global_fci.e_tot - restric_driver._global_ks.energy_nuc()
print(rks)
rbuilder = HamiltonianBuilder(restric_driver._global_ks, 0, "jordan_wigner")
rham = rbuilder.build()
rdiag, _ = sp.sparse.linalg.eigsh(get_sparse_operator(rham), k=1, which="SA")
print(rdiag[0])
# print(rham, rdiag)

-84.29484027700339
-84.2948402770431


The driver requires two h_core matrices, one for each spin, but there is only one in the KS object, so we need to double this!

In [3]:
# Sectond check unrestricted
import numpy as np
uks = unrestric_driver._global_fci.e_tot - unrestric_driver._global_ks.energy_nuc()
print(uks)

old_hcore = unrestric_driver._global_ks.get_hcore()
unrestric_driver._global_ks.get_hcore = lambda *args: np.array([old_hcore, old_hcore])

ubuilder = HamiltonianBuilder(unrestric_driver._global_ks, 0, "jordan_wigner")
u_one_body = ubuilder._one_body_integrals

uham = ubuilder.build()
udiag, _ = sp.sparse.linalg.eigsh(get_sparse_operator(uham), k=1, which="SA")
print(udiag[0])

-84.29484027700333
-84.29484027705551


Now lets check with charge and spin not equal to zero!

In [5]:
driver = NbedDriver(
    geometry=args["geometry"],
    n_active_atoms=args["n_active_atoms"],
    basis=args["basis"],
    xc_functional=args["xc_functional"],
    projector=args["projector"],
    localization=args["localization"],
    convergence=args["convergence"],
    savefile=args["savefile"],
    run_ccsd_emb=args["run_ccsd_emb"],
    run_fci_emb=args["run_fci_emb"],
    force_unrestricted=True,
    charge=1,
    spin=1,
)

ks = driver._global_fci.e_tot - driver._global_ks.energy_nuc()
print(ks)

old_hcore = driver._global_ks.get_hcore()
driver._global_ks.get_hcore = lambda *args: np.array([old_hcore, old_hcore])

builder = HamiltonianBuilder(driver._global_ks, 0, "jordan_wigner")
ham = builder.build()
diag, _ = sp.sparse.linalg.eigsh(get_sparse_operator(ham), k=1, which="SA")
print(diag[0])

-83.97614236525206
-84.57115111164894


In [ ]:
def _unrestricted_spinorb_from_spatial(one_body_integrals, two_body_integrals):
    n_qubits = one_body_integrals[0].shape[0] * 2
    
    # Initialize Hamiltonian coefficients.
    one_body_coefficients = np.zeros((n_qubits, n_qubits))
    two_body_coefficients = np.zeros((n_qubits, n_qubits, n_qubits, n_qubits))

    # Loop through integrals.
    for p in range(n_p):
        for q in range(n_q):

            # Populate 1-body coefficients. Require p and q have same spin.
            one_body_coefficients[2 * p, 2 * q] = one_body_integrals[0, p, q]
            # one_body_coefficients[2 * p, 2 * q] = one_body_integrals[1, p, q]

            # one_body_coefficients[2 * p + 1, 2 * q + 1] = one_body_integrals[0, p, q]
            one_body_coefficients[2 * p + 1, 2 * q + 1] = one_body_integrals[1, p, q]

            # Continue looping to prepare 2-body coefficients.
            for r in range(n_qubits // 2):
                for s in range(n_qubits // 2):

                    # Mixed spin
                    two_body_coefficients[
                        2 * p, 
                        2 * q + 1, 
                        2 * r + 1, 
                        2 * s
                    ] = two_body_integrals[0][p, q, r, s]

                    two_body_coefficients[
                        2 * p + 1, 
                        2 * q, 
                        2 * r, 
                        2 * s + 1
                    ] = two_body_integrals[1][p, q, r, s]

                    # Same spin
                    two_body_coefficients[
                        2 * p, 
                        2 * q, 
                        2 * r, 
                        2 * s
                    ] = two_body_integrals[0][p, q, r, s]

                    two_body_coefficients[
                        2 * p + 1, 
                        2 * q + 1, 
                        2 * r + 1, 
                        2 * s + 1
                    ] = two_body_integrals[1][p, q, r, s]

    # Truncate.
    one_body_coefficients[np.absolute(one_body_coefficients) < EQ_TOLERANCE] = 0.0
    two_body_coefficients[np.absolute(two_body_coefficients) < EQ_TOLERANCE] = 0.0

    return one_body_coefficients, two_body_coefficients

In [9]:
from openfermion.chem.molecular_data import spinorb_from_spatial

obi = builder._one_body_integrals
tbi = builder._two_body_integrals

print(obi.shape)
print(tbi.shape)

spinorb_from_spatial(obi, tbi)

(2, 7, 7)
(2, 7, 7, 7, 7)


ValueError: setting an array element with a sequence.

### FCI and diagonalisation results match for systems with closed shells and no spin, however the results __don't match for charge and spin = 1__


## 3.

This is the trickiest bit
- We can first check that the general behaviour is correct, what do we expect for the +1 and -1 ions of a molecule?
- Check results against physical limits, for instance if any are below the ground state these need to be explained.
- Find published energies of some example ions and compare to these.